In [1]:
using TensorKit
import TensorKit.×
include("./iPEPS_fSU2/iPEPS.jl")
include("./CTMRG_fSU2/CTMRG.jl")
@show pkgversion(TensorKit)
using JLD2

pkgversion(TensorKit) = v"0.12.3"


In [2]:
# pspace = GradedSpace{fSU₂}((-1, 0) => 1, (0, 1 // 2) => 1, (1, 0) => 1)
pspace = GradedSpace{fSU₂}((0 => 2), (1//2 => 1))
aspacel = GradedSpace{fSU₂}((0 => 1), (1//2 => 1))
aspacet = GradedSpace{fSU₂}((0 => 1), (1//2 => 2))
aspacer = GradedSpace{fSU₂}((0 => 2), (1//2 => 2))
aspaceb = GradedSpace{fSU₂}((0 => 1), (1//2 => 1), (1 => 1))
# aspacel = aspacer = aspacet = aspaceb = pspace
@show pspace
nothing

pspace = Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1)


In [ ]:
a = id(pspace)
@tensor contractcheck=true b[] := a[x, in] * a[in, x]
@show scalar(b)

@show maximum(convert(Array, a))
@show minimum(convert(Array, a))
@show one(eltype(a))
@show one(mapreduce(eltype, promote_type, storagetype.(a)))
nothing

In [38]:
t1 = TensorMap(randn, ComplexF64, pspace ⊗ pspace, pspace ⊗ pspace)
# t1 = TensorMap(randn, Float64, ℂ^2 ⊗ ℂ^2, ℂ^2 ⊗ ℂ^2)

@show scalartype(t1)
D, V = eig(t1, ((1, 2), (3, 4)))
@show space(D)
@show space(V)
@show scalartype(D)
@show scalartype(V)
@assert permute(t1, ((1, 2), (3, 4))) * V ≈ V * D
@assert permute(t1, ((1, 2), (3, 4))) * V ≈ D
@show D
@show V' * permute(t1, ((1, 2), (3, 4))) * V


@assert V * V' == id(pspace ⊗ pspace)
@tensor contractcheck=true t1p[a, b; e, f] := V[a, b, c] * D[c, d] * V'[d, e, f]
@show norm(t1 - t1p)
@assert t1 ≈ t1p

scalartype(t1) = ComplexF64
space(D) = Vect[FermionSpin]((0, 0)=>5, (1, 0)=>1, (1/2, 1)=>4) ← Vect[FermionSpin]((0, 0)=>5, (1, 0)=>1, (1/2, 1)=>4)
space(V) = (Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1) ⊗ Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1)) ← Vect[FermionSpin]((0, 0)=>5, (1, 0)=>1, (1/2, 1)=>4)
scalartype(D) = Float64
scalartype(V) = ComplexF64


AssertionError: AssertionError: permute(t1, ((1, 2), (3, 4))) * V ≈ V * D

In [ ]:
a = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
@show space(a)  # [l, t, p; r, b]
@show space(a') # [r, b; l, t, p]

permute!(a, a, ((1,2), (3,4,5)))
@show space(a)
nothing

In [ ]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet

# A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
# Ms = [A, A, A, A]
# ipeps = iPEPS(Ms, 2, 2)
# Abar = bar(A)
# @show space(Abar)
# gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
# @show space(gate1)
# gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
# @show space(gate2)
# gate3 = swap_gate(space(A)[4], space(A)[5])
# @show space(gate3)
# gate4 = swap_gate(space(A)[1], space(A)[2])
# @show space(gate4)
# gate5 = swap_gate(space(A)[1], space(Abar)[2])
# @show space(gate5)
@assert pspace isa ElementarySpace
@show fieldnames(GradedSpace)

In [ ]:
pspace = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspacel = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspacer = aspacel
aspacet = GradedSpace{fSU₂}((0 => 2), (1 // 2 => 1))
aspaceb = aspacet

Lx = 2
Ly = 2
Ms = Vector{TensorMap}(undef, Lx * Ly)
for ii in 1:Lx*Ly
    Ms[ii] = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
    Ms[ii] = Ms[ii] / norm(Ms[ii])
end
@assert Ms[1] != Ms[2]
ipeps = iPEPS(Ms, Lx, Ly)

envs = iPEPSenv(ipeps)
χ = 10
Nit = 2
nothing

In [ ]:
save("/home/tcmp2/JuliaProjects/testSave.jld2", "ipeps", ipeps, "envs", envs)

In [ ]:
ipeps, envs = load("/home/tcmp2/JuliaProjects/testSave.jld2", "ipeps", "envs")
@show ipeps.Lx
@show envs.Ly
nothing

In [1]:
using TensorKit
include("./iPEPS_fSU2/iPEPS.jl")
include("./CTMRG_fSU2/CTMRG.jl")
include("./models/Heisenberg_U1.jl")
include("./simple_update_fSU2/simple_update.jl")
include("./Cal_Obs_fSU2/Cal_Obs.jl")

_2siteObs_diagSite (generic function with 1 method)

In [2]:
ipeps, envs, para = load("/home/tcmp2/JuliaProjects/testDiagObs.jld2", "ipeps", "envs", "para")
Obsdiag1 = Cal_Obs_2site(ipeps, envs, ["hijNNN", "SzSz", "SpSm"], para; site1=[1, 1], site2=[2, 2], get_op=get_op_Heisenberg)
Obsdiag1 = Cal_Obs_2site(ipeps, envs, ["hijNNN", "SzSz", "SpSm"], para; site1=[1, 2], site2=[2, 1], get_op=get_op_Heisenberg)

Calculating ["hijNNN", "SzSz", "SpSm"] at site [1, 1] and site [2, 2]
Calculating ["hijNNN", "SzSz", "SpSm"] at site [1, 2] and site [2, 1]


Dict{String, Float64} with 3 entries:
  "hijNNN" => -0.125789
  "SzSz"   => -0.249729
  "SpSm"   => -0.00184846